- create hmm prototype

In [ ]:
from hmmfunc import MonoPhone

if __name__ == '__main__':
    phones_file = '# the file where list of phones are saved'
    num_states = 3
    num_dims = 13
    prob_loop = 0.7 # initial value
    out_dir = '# the directory you want to save hmm prototype'
    
    # create phone_list
    phone_list = []
    with open(phones_file, mode = 'r') as f:
        for line in f:
            phone = line.split()[0]
            phone_list.append(phone)
    
    # create MonoPhone instance
    hmm = MonoPhone()
    # create hmm prototype
    hmm.make_proto(phone_list, num_states, num_dims, prob_loop)
    # save the prototype
    out_hmm = os.path.join(out_dir, 'hmm_proto')
    hmm.save_hmm(out_hmm)

In [ ]:
def load_hmm(self, filename):
        ''' json形式のHMMファイルを読み込む
        filename: 読み込みファイル名
        '''
        # JSON形式のHMMファイルを読み込む
        with open(filename, mode='r') as f:
            hmmjson = json.load(f)

        # 辞書の値を読み込んでいく
        self.num_phones = hmmjson['num_phones']
        self.num_states = hmmjson['num_states']
        self.num_mixture = hmmjson['num_mixture']
        self.num_dims = hmmjson['num_dims']

        # 音素情報の読み込み
        self.phones = []
        for p in range(self.num_phones):
            hmms = hmmjson['hmms'][p]
            self.phones.append(hmms['phone'])

        #遷移確率の読み込み
        # 音素番号p, 状態番号s の遷移確率は
        # trans[p][s] = [loop, next]
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                hmm = hmms['hmm'][s]
                # 遷移確率の読み込み
                tmp_trans = np.array(hmm['trans'])
                # 総和が1になるよう正規化
                tmp_trans /= np.sum(tmp_trans)
                # 対数に変換
                for i in [0, 1]:
                    tmp_trans[i] = np.log(tmp_trans[i]) \
                        if tmp_trans[i] > self.ZERO \
                        else self.LZERO
                tmp_p.append(tmp_trans)
            # self.transに追加
            self.trans.append(tmp_p)

        # 正規分布パラメータの読み込み
        # 音素番号p, 状態番号s, 混合要素番号m
        # の正規分布はpdf[p][s][m]でアクセスする
        # pdf[p][s][m] = gaussian
        self.pdf = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                tmp_s = []
                hmm = hmms['hmm'][s]
                for m in range(self.num_mixture):
                    gmm = hmm['gmm'][m]
                    # 重み，平均，分散，gConstを取得
                    weight = gmm['weight']
                    mu = np.array(gmm['mean'])
                    var = np.array(gmm['variance'])
                    gconst = gmm['gConst']
                    # 正規分布を作成
                    gaussian = {'weight': weight, 
                                'mu': mu, 
                                'var': var,
                                'gConst': gconst}
                    tmp_s.append(gaussian)
                tmp_p.append(tmp_s)
            # self.pdfに追加
            self.pdf.append(tmp_p)